In [1]:
import pandas as pd
from abc import ABC, abstractmethod
from random import randint

In [2]:
class Handler(ABC):
    """
    Интерфейс обработчика объявляет метод построения цепочки обработчиков и метод для обработки заказа.
    """
    @abstractmethod
    def set_next(self, handler):
        pass

    @abstractmethod
    def process(self, code, quant, cust):
        pass

In [3]:
class AbstractHandler(Handler):
    """ 
    Базовый класс обработчика, объявляющий поведение цепочки по умолчанию.
    """
    next_handler = None
    def set_next(self, handler):
        """
        Устанавливает, какой обработчик будет следующим пытаться обработать заказ.
        handler -- следующий обработчки
        type handler -- Handler
        """
        self.next_handler = handler
        return handler
    
    @abstractmethod
    def process(self, code, quant, cust):
        """
        Обработка заказа по умолчанию.
        Возвращает результат обработки следующего обработчика, если он есть в цепочке,
        или None, если это последний обработчик в цепочке.
        code -- код заказа
        type code - количество товара
        type quant -- int
        cust -- код заказчика
        type cust -- int
        """
        if self.next_handler:
            return self.next_handler.process(code, quant, cust)
        return None

In [4]:
#пусть оптовым считается заказ, если число товара > 7. Тогда номер заказа начинается с "o"
#проверяем, что в заказе нет ошибки и такой товар в магазине есть(код товара = 1..2380)

class Handler_Opt(AbstractHandler):
    """
    Обработчик, проверяющий, является ли заказ оптовым.
    """
    def process(self, code, quant, cust):
        """
        Процесс обработки: является ли заказ оптовым? Если да, то есть ли товар в магазине?
        code -- код заказа
        type code - количество товара
        type quant -- int
        cust -- код заказчика
        type cust -- int        
        """
        if quant > 7:
            if code > 0 and code < 2381:
                k = randint(1, 99)
                print("order number: o{num}, ".format(num="0{x}".format(x=k) if k//10==0 else k), "product code: {x}, ".format(x=code), "quantity: {x}, ".format(x=quant), "customer ID: {x}".format(x=cust))
            else:
                print("This order is incorrect :c")
        else:
            return super().process(code, quant, cust)

In [5]:
#пусть vip-клиентами считаются те, чьи номера начинаются с 9. Тогда их надо обслужить быстрее -> номер заказа начинается с "f"
#проверяем, что в заказе нет ошибки и такой товар в магазине есть(код товара = 1..2380)

class Handler_Fast(AbstractHandler):
    """
    Обработчик, проверяющий, является ли заказчик vip-клиентом.
    """
    def process(self, code, quant, cust):
        """
        Процесс обработки: является ли заказчик vip? Если да, то есть ли товар в магазине?
        code -- код заказа
        type code - количество товара
        type quant -- int
        cust -- код заказчика
        type cust -- int        
        """
        if str(cust)[0] == '9':
            if code > 0 and code < 2381:
                k = randint(1, 99)
                print("order number: f{num}, ".format(num="0{x}".format(x=k) if k//10==0 else k), "product code: {x}, ".format(x=code), "quantity: {x}, ".format(x=quant), "customer ID: {x}".format(x=cust))
            else:
                print("This order is incorrect :c")
        else:
            return super().process(code, quant, cust)

In [6]:
#пусть на определенные виды товаров(их коды заканчиваются на 9) распространяется акция: бесплатная доставка -> 
#номер заказа начинается с "w"
#проверяем, что в заказе нет ошибки и такой товар в магазине есть(код товара = 1..2380)

class Handler_Sale(AbstractHandler):
    """
    Обработчик, проверяющий, является ли заказанный товар акционным.
    """
    def process(self, code, quant, cust):
        """
        Процесс обработки: является ли товар акционным? Если да, то есть ли он в магазине?
        code -- код заказа
        type code - количество товара
        type quant -- int
        cust -- код заказчика
        type cust -- int        
        """
        if cust % 10 == 9:
            if code > 0 and code < 2381:
                k = randint(1, 99)
                print("order number: w{num}, ".format(num="0{x}".format(x=k) if k//10==0 else k), "product code: {x}, ".format(x=code), "quantity: {x}, ".format(x=quant), "customer ID: {x}".format(x=cust))
            else:
                print("This order is incorrect :c")
        else:
            return super().process(code, quant, cust)

In [7]:
#пусть номер стандартных заказов не сопровождается отличительной буквой
#проверяем, что в заказе нет ошибки и такой товар в магазине есть(код товара = 1..2380)

class Handler_Usual(AbstractHandler):
    """
    Обработчик стандартных заказов.
    """
    def process(self, code, quant, cust):
        """
        Процесс обработки: ладно, ничего особенного в заказе нет. Но товар-то есть в магазине?
        code -- код заказа
        type code - количество товара
        type quant -- int
        cust -- код заказчика
        type cust -- int        
        """
        if code > 0 and code < 2381:
            k = randint(1, 99)
            print("order number: {num}, ".format(num=k), "product code: {x}, ".format(x=code), "quantity: {x}, ".format(x=quant), "customer ID: {x}".format(x=cust))
        else:
            print("This order is incorrect :c")

In [9]:
opt = Handler_Opt()
fast = Handler_Fast()
sale = Handler_Sale()
usual = Handler_Usual()
"""
Задаем последовательность обработки заказов
"""
opt.set_next(fast).set_next(sale).set_next(usual)

orders = []
for i in range(25):
    orders.append((randint(-10, 3000), randint(1, 10), randint(1, 10000)))
for current in orders:
    opt.process(current[0], current[1], current[2])
    

order number: 25,  product code: 52,  quantity: 5,  customer ID: 198
This order is incorrect :c
order number: 59,  product code: 2144,  quantity: 1,  customer ID: 2454
order number: o36,  product code: 145,  quantity: 10,  customer ID: 33
order number: o08,  product code: 259,  quantity: 8,  customer ID: 8114
order number: o53,  product code: 2004,  quantity: 8,  customer ID: 3894
order number: 81,  product code: 2362,  quantity: 3,  customer ID: 4196
order number: w79,  product code: 1773,  quantity: 1,  customer ID: 6419
This order is incorrect :c
order number: o86,  product code: 2171,  quantity: 8,  customer ID: 7808
order number: w37,  product code: 387,  quantity: 1,  customer ID: 4599
This order is incorrect :c
order number: 24,  product code: 662,  quantity: 1,  customer ID: 4960
order number: o01,  product code: 1643,  quantity: 8,  customer ID: 5714
order number: 11,  product code: 2087,  quantity: 5,  customer ID: 1295
order number: o84,  product code: 254,  quantity: 10,  c